In [1]:
import pandas as pd

### Retour sur la base de donnée des fréquentation des musées.

In [4]:
# https://www.data.gouv.fr/fr/datasets/frequentation-des-musees-de-france/
df = pd.read_csv(
    "https://www.data.gouv.fr/fr/datasets/r/81e9e4c0-296d-46b0-9c49-9d092c09dcc5")

On voit que l'information qui concerne la fréquentation est contenu dans la colonne `stats`, mais dans un format difficilement exploitable.

`payant:0;gratuit:0;label-date:01/02/2003`

Comment faire pour utiliser cette information ? 

Avec la methode de string accessor `.str` ! 

In [8]:
df.stats.str

In [10]:
# Remarque si la série n'est pas de type objet ou string, pas de méthode str !
df.lat.str

AttributeError: Can only use .str accessor with string values!

In [15]:
df.tags.str[:6]

0       label:
1       label:
2       label:
3       label:
4       label:
         ...  
1156    label:
1157    label:
1158    label:
1159    label:
1160    label:
Name: tags, Length: 1161, dtype: object

In [17]:
df.tags.str[:6].value_counts()

label:    1150
unlabe      11
Name: tags, dtype: int64

Str permet de manipuler chaque ligne de la colonne comme si c'était une string.

In [18]:
df.tags.str[:6].value_counts()

label:    1150
unlabe      11
Name: tags, dtype: int64

In [21]:
df.tags.str.split(":").str[1].value_counts()

musee de france    1161
Name: tags, dtype: int64

### Exercice: 
    - créez deux colonnes qui contient respectivement le nombre d'entrée en gratuit et en payant par musée
    - donnez le pourcentage d'entrée gratuite par musée, puis ce ratio moyen entre les musées, puis le ratio moyen général.

In [47]:
df.stats.str.split(";", expand=True)

,0,1,2
0,payant:0,gratuit:0,label-date:01/02/2003
1,payant:1644,gratuit:2174,label-date:01/02/2003
2,payant:1509,gratuit:2397,label-date:01/02/2003
3,payant:72733,gratuit:21431,label-date:23/10/2009
4,payant:0,gratuit:0,label-date:01/02/2003
...,...,...,...
1156,payant:0,gratuit:16705,label-date:06/01/2002
1157,payant:0,gratuit:0,label-date:01/02/2003
1158,payant:232,gratuit:963,label-date:01/02/2003
1159,payant:0,gratuit:425,label-date:01/02/2003


In [48]:
df[["payant", "gratuit", "date"]] = df.stats.str.split(";", expand=True)

In [49]:
df["payant"] = df.payant.str.split(":").str[1].astype("int")
df["gratuit"] = df.gratuit.str.split(":").str[1].astype("int")

In [65]:
df_result = df[["payant", "gratuit"]].assign(
    part_payant=lambda x: (x["payant"]/(x["gratuit"]+x["payant"]))*100
)
df_result = df_result.append(df_result.sum().rename('Total'), ignore_index=False
                             ).append(df_result.iloc[:-1].mean().rename("Moyenne"), ignore_index=False)

In [70]:
df_result.loc["Moyenne"]

payant           32442.400000
gratuit          24525.124138
part_payant         44.323997
Moyenne_total             NaN
Name: Moyenne, dtype: float64

In [72]:
(df.payant/df.gratuit)

0            NaN
1       0.756210
2       0.629537
3       3.393822
4            NaN
          ...   
1156    0.000000
1157         NaN
1158    0.240914
1159    0.000000
1160    5.963443
Length: 1161, dtype: float64

In [79]:
df_result.loc["Moyenne", "Moyenne_total"] = (df_result.payant /
                                             (df_result.gratuit+df_result.payant)
                                             ).loc["Moyenne"]

In [80]:
df_result.fillna("")

,payant,gratuit,part_payant,Moyenne_total
0,0.0,0.000000e+00,,
1,1644.0,2.174000e+03,43.059193,
2,1509.0,2.397000e+03,38.632873,
3,72733.0,2.143100e+04,77.240771,
4,0.0,0.000000e+00,,
...,...,...,...,...
1158,232.0,9.630000e+02,19.414226,
1159,0.0,4.250000e+02,0.0,
1160,59378.0,9.957000e+03,85.639288,
Total,37692562.0,2.845910e+07,45517.735747,
